In [2]:
# Import library-library
import os
import glob
import string

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

import mysql.connector
from mysql.connector import errorcode

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
excluded_words = ["tempat", "waktu", "gempa", "banjir"]

NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

In [4]:
def preprocessing(berita):
    # Preprocessing
    s = str(berita)
    s = s.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]

    # Stopwords checking
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

In [5]:
# df =pd.read_csv("news.csv")
# df.info()
# df.head()
dataframe = []

try:
        cnx = mysql.connector.connect(user='admin', password='admin', database = 'Petakabar')
        cursor = cnx.cursor()

        ambildata = ("SELECT berita_title, berita_date, berita_desc, berita_source FROM berita where berita_topik_id = 1")
        cursor.execute(ambildata)
        dataa = cursor.fetchall()
        for row in dataa:
            dataframe.append(row)
        
except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
else:
    cursor.close()
    cnx.close()

In [6]:
import pandas as pd
writer = pd.DataFrame(dataframe, columns=[
                    'Title','Date','Description', 'Source'], index=None)
writer.to_csv('news.csv', index=False, sep=',')

In [7]:
df =pd.read_csv("news.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2773 entries, 0 to 2772
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        2773 non-null   object
 1   Date         2773 non-null   object
 2   Description  2773 non-null   object
 3   Source       2773 non-null   object
dtypes: object(4)
memory usage: 86.8+ KB


,Title,Date,Description,Source
0,Bagaimana Proses Terjadinya Gempa Bumi?,"Kamis, 15 Des 2022 17:02 WIB","\- Saat gempa bumi terjadi, tanah dan apa saja...",www.kompas.com
1,"Rumah Tinggal di Bangka Jaksel Terbakar, 3 Uni...","Senin, 26 Des 2022 22:01 WIB",\- Kebakaran melanda rumah tinggal Jalan Bangk...,www.kompas.com
2,Tim SAR Ungkap Cara Temukan Korban Gempa Cianj...,"Minggu, 27 Nov 2022 15:52 WIB",\- Tim SAR (search and rescue) masih terus mel...,www.kompas.com
3,Jalan Nasional Cibadak-Palabuhanratu Tertimbun...,"Selasa, 13 Des 2022 08:17 WIB",Hujan deras yang mengguyur wilayah Sukabumi se...,www.kompas.com
4,"Ketika Petir Menyambar Meteran Listrik, Puluha...","Senin, 19 Des 2022 06:05 WIB",\- Kebakaran melanda 25 unit rumah kontrakan d...,www.kompas.com


In [8]:
column = ['Title', 'Date', 'Description', 'Source']
df_total = pd.DataFrame()

df = pd.read_csv("news.csv")
df = df.dropna(subset=['Description'], axis=0)
df.sort_values("Description", inplace = True)
df.drop_duplicates(subset ="Description", keep = False, inplace = True)

df.head()
df_train_unprocessed, df_test_unprocessed = train_test_split(df,test_size=0.1)

In [9]:
# df_first_1000 = df.head(1000)
# len(df_first_1000)
df_train, df_test = train_test_split(df,test_size=0.1)
len(df_train)


2495

In [10]:
df_train.to_csv("corpus/df_train_unprerocessed.csv" , index = None, header=True)
df_test.to_csv("corpus/df_test_unprerocessed.csv" , index = None, header=True)

In [11]:
desc_text_test_unprocessed = []

for x in range(0, df_test_unprocessed.shape[0]):
  desc_text_test_unprocessed.append(df_test_unprocessed.iloc[x, -2])

joblib.dump(desc_text_test_unprocessed, "corpus/model/desc_text_test_unprocessed.pkl")

['corpus/model/desc_text_test_unprocessed.pkl']

In [12]:
# for x in range(0, df.shape[0]):
#     text = preprocessing(df.iloc[x, -2])
#     df.iloc[x, -2] = (' '.join(text)) 

desc_text_train_unpreprocessed = []
desc_text_test_unpreprocessed = []

for x in range(0, df_train.shape[0]):
  desc_text_train_unpreprocessed.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test_unpreprocessed.append(df_test.iloc[x, -2])

In [13]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test_unpreprocessed, "corpus/model/desc_text_test_unpreprocessed.pkl")
joblib.dump(desc_text_train_unpreprocessed, "corpus/model/desc_text_train_unpreprocessed.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total_unpreprocessed.csv', index=False)
df_test.to_csv('corpus/dataset/df_test_unpreprocessed.csv', index=False)
df_train.to_csv('corpus/dataset/df_train_unpreprocessed.csv', index=False)


In [14]:
for x in range(0, df.shape[0]):
    text = preprocessing(df.iloc[x, -2])
    df.iloc[x, -2] = (' '.join(text)) 

In [15]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [16]:
# print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

# print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

Panjang Dokumen Test :  278
------------------------------------------------------------------------------------------
Panjang Dokumen Train :  2495


In [17]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, "corpus/model/desc_text_test.pkl")
joblib.dump(desc_text_train, "corpus/model/desc_text_train_new.pkl") #INI

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total_new.csv', index=False) #INI
df_test.to_csv('corpus/dataset/df_test.csv', index=False)
df_train.to_csv('corpus/dataset/df_train.csv', index=False)


In [18]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

tfidf_test_matrix
(278, 8741)
tfidf_train_matrix
(2495, 29671)


In [19]:
joblib.dump(tfidf_vectorizer, "corpus/vectorizer/vectorizer_new.pkl") #INI
joblib.dump(tfidf_test_matrix, "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "corpus/matrix/tfidf_train_new.pkl") #INI

['corpus/matrix/tfidf_train_new.pkl']

### Pencarian 

In [20]:
df_total = pd.read_csv('corpus/dataset/df_total_new.csv')
df_total = df_total[pd.notnull(df_total['Description'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load('corpus/model/desc_text_train_new.pkl')
document_text_test= joblib.load('corpus/model/desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2773 entries, 0 to 2772
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        2773 non-null   object
 1   Date         2773 non-null   object
 2   Description  2773 non-null   object
 3   Source       2773 non-null   object
dtypes: object(4)
memory usage: 108.3+ KB
None
------------------------------------------------------------------------------------------
278
2495


In [21]:
kueri='gempa'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "corpus/matrix/tfidf_train_new.pkl" )
tfidf_vectorizer = joblib.load( "corpus/vectorizer/vectorizer_new.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])

['gempa']


Top 10 most similar documents in corpus:
No ID Dokumen  :  2310
Tanggal        :  Selasa, 29 Nov 2022 08:26 WIB
Isi berita     :  tim gabungan pencarian sembilan korban gempa bumi cianjur pencarian kesembilan dinyatakan hilang pencarian selasa 29 11 2022 pencarian difokuskan wilayah kecamatan cugenang tim gabungan melanjutkan pencarian korban gempa bumi cianjur korban pencarian 9 kepala kantor bandung jumaril keterangan tertulis diterima jumaril pencarian kesembilan dibagi tim meliputi worksite 1 warung sate sinta 36 personil 5 anjing pelacak 4 life detector worksite 2 03 rw 01 desa cijendil 74 personil 4 anjing pelacak 4 life detector worksite 3 kampung cicadas 81 personil 2 anjing pelacak 4 life detector mengimbau aparatur desa wilayah terdampak gempa bumi pendataan warga warganya aman korban menghimbau aparat desa warga kehilangan anggota keluarga kerabatnya melaporkan nomor layanan pengaduan 0 813 1616 3355 kegiatan dropping dukungan logistik terpal paket keluarga paket